In [ ]:
import cv2
import numpy as np
import pickle
import os
from pathlib import Path

class FaceRecognitionSystem:
    def __init__(self):
        """Инициализация системы распознавания лиц"""
        # Загрузка Haar Cascade для детекции лиц
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        
        # LBPH распознаватель (Local Binary Patterns Histograms)
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        
        # Путь для сохранения модели и данных
        self.model_path = 'face_recognition_model.yml'
        self.labels_path = 'face_labels.pkl'
        
        # Словарь: ID -> имя
        self.labels_dict = {}
        self.next_id = 0
        
        # Загрузка существующей модели, если есть
        self.load_model()
        
    def detect_faces(self, frame):
        """Детекция лиц на кадре"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(100, 100)
        )
        return faces, gray
    
    def capture_face_data(self, name, num_samples=100):
        """Захват образцов лица для обучения
        
        Args:
            name: имя человека
            num_samples: количество кадров для захвата
        """
        cap = cv2.VideoCapture(0)
        
        # Проверка, есть ли уже это имя
        if name in self.labels_dict.values():
            person_id = [k for k, v in self.labels_dict.items() if v == name][0]
        else:
            person_id = self.next_id
            self.labels_dict[person_id] = name
            self.next_id += 1
        
        samples = []
        count = 0
        
        print(f"\nЗахват данных для '{name}'...")
        print(f"Смотрите в камеру. Будет захвачено {num_samples} кадров.")
        print("Медленно поворачивайте голову для лучшего результата.")
        
        while count < num_samples:
            ret, frame = cap.read()
            if not ret:
                break
            
            faces, gray = self.detect_faces(frame)
            
            for (x, y, w, h) in faces:
                # Рисуем прямоугольник
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Извлекаем область лица
                face_roi = gray[y:y+h, x:x+w]
                
                # Сохраняем образец
                samples.append((face_roi, person_id))
                count += 1
                
                # Показываем прогресс
                progress = f"{count}/{num_samples}"
                cv2.putText(frame, progress, (x, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Показываем информацию
            info_text = f"Capturing: {name} ({count}/{num_samples})"
            cv2.putText(frame, info_text, (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow('Training - Press ESC to cancel', frame)
            
            if cv2.waitKey(1) & 0xFF == 27:  # ESC для отмены
                print("Захват отменён")
                cap.release()
                cv2.destroyAllWindows()
                return False
        
        cap.release()
        cv2.destroyAllWindows()
        
        # Обучение модели на новых данных
        if samples:
            print(f"Обучение модели на {len(samples)} образцах...")
            faces_data = [sample[0] for sample in samples]
            labels_data = [sample[1] for sample in samples]
            
            # Если модель уже обучена, обновляем её
            if os.path.exists(self.model_path):
                self.recognizer.update(faces_data, np.array(labels_data))
            else:
                self.recognizer.train(faces_data, np.array(labels_data))
            
            self.save_model()
            print(f"✓ Обучение завершено! {name} добавлен в базу.")
            return True
        
        return False
    
    def recognize_faces(self):
        """Распознавание лиц в реальном времени"""
        cap = cv2.VideoCapture(0)
        
        print("\n=== РЕЖИМ РАСПОЗНАВАНИЯ ===")
        print("ESC - выход")
        print("Клавиши для обучения:")
        print("  '1' - добавить нового человека")
        print("  'r' - переобучить на текущем лице")
        
        current_name_for_training = ""
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            faces, gray = self.detect_faces(frame)
            
            for (x, y, w, h) in faces:
                # Рисуем зелёный квадрат вокруг лица
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
                
                # Распознавание
                face_roi = gray[y:y+h, x:x+w]
                
                if os.path.exists(self.model_path) and self.labels_dict:
                    try:
                        label, confidence = self.recognizer.predict(face_roi)
                        
                        # Confidence: чем меньше, тем лучше (0 = идеальное совпадение)
                        # Обычно < 50 - хорошее совпадение, > 80 - неизвестное лицо
                        if confidence < 70:
                            name = self.labels_dict.get(label, "Unknown")
                            confidence_percent = int(100 - confidence)
                            text = f"{name} ({confidence_percent}%)"
                            color = (0, 255, 0)  # Зелёный
                        else:
                            text = "Unknown"
                            color = (0, 0, 255)  # Красный
                    except:
                        text = "Unknown"
                        color = (0, 0, 255)
                else:
                    text = "No trained data"
                    color = (0, 165, 255)  # Оранжевый
                
                # Текст справа от квадрата с фоном
                text_x = x + w + 10
                text_y = y + 30
                
                # Размер текста для фона
                (text_width, text_height), _ = cv2.getTextSize(
                    text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2
                )
                
                # Рисуем фон для текста
                cv2.rectangle(frame, 
                             (text_x - 5, text_y - text_height - 5),
                             (text_x + text_width + 5, text_y + 5),
                             (0, 0, 0), -1)
                
                # Рисуем текст
                cv2.putText(frame, text, (text_x, text_y),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            
            # Информация на экране
            info = f"Faces detected: {len(faces)} | Press '1' to add person"
            cv2.putText(frame, info, (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Показываем список известных лиц
            y_offset = 60
            if self.labels_dict:
                cv2.putText(frame, "Known faces:", (10, y_offset),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
                y_offset += 25
                for person_id, person_name in self.labels_dict.items():
                    cv2.putText(frame, f"  - {person_name}", (10, y_offset),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
                    y_offset += 20
            
            cv2.imshow('Face Recognition - ESC to exit', frame)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == 27:  # ESC
                break
            elif key == ord('1'):  # Добавить нового человека
                cap.release()
                cv2.destroyAllWindows()
                
                name = input("\nВведите имя нового человека: ").strip()
                if name:
                    self.capture_face_data(name)
                
                # Перезапускаем камеру
                cap = cv2.VideoCapture(0)
        
        cap.release()
        cv2.destroyAllWindows()
    
    def save_model(self):
        """Сохранение модели и меток"""
        self.recognizer.save(self.model_path)
        with open(self.labels_path, 'wb') as f:
            pickle.dump({
                'labels_dict': self.labels_dict,
                'next_id': self.next_id
            }, f)
        print(f"Модель сохранена: {self.model_path}")
    
    def load_model(self):
        """Загрузка модели и меток"""
        if os.path.exists(self.model_path) and os.path.exists(self.labels_path):
            try:
                self.recognizer.read(self.model_path)
                with open(self.labels_path, 'rb') as f:
                    data = pickle.load(f)
                    self.labels_dict = data['labels_dict']
                    self.next_id = data['next_id']
                print(f"✓ Модель загружена. Известно лиц: {len(self.labels_dict)}")
                for pid, name in self.labels_dict.items():
                    print(f"  - {name}")
            except Exception as e:
                print(f"Ошибка загрузки модели: {e}")
    
    def delete_person(self, name):
        """Удаление человека из базы (требует переобучения)"""
        person_id = None
        for pid, pname in self.labels_dict.items():
            if pname == name:
                person_id = pid
                break
        
        if person_id is not None:
            del self.labels_dict[person_id]
            print(f"✓ {name} удалён из базы")
            print("⚠ Требуется переобучение модели с нуля")
        else:
            print(f"Человек '{name}' не найден в базе")
    
    def list_known_faces(self):
        """Список всех известных лиц"""
        if not self.labels_dict:
            print("База пуста. Нет обученных лиц.")
        else:
            print("\n=== ИЗВЕСТНЫЕ ЛИЦА ===")
            for person_id, name in self.labels_dict.items():
                print(f"ID {person_id}: {name}")


def main():
    """Главное меню приложения"""
    system = FaceRecognitionSystem()
    
    while True:
        print("\n" + "="*50)
        print("СИСТЕМА РАСПОЗНАВАНИЯ ЛИЦ")
        print("="*50)
        print("1. Запустить распознавание в реальном времени")
        print("2. Добавить нового человека (обучение)")
        print("3. Показать список известных лиц")
        print("4. Удалить человека из базы")
        print("5. Выход")
        print("="*50)
        
        choice = input("Выберите действие (1-5): ").strip()
        
        if choice == '1':
            system.recognize_faces()
        
        elif choice == '2':
            name = input("Введите имя человека: ").strip()
            if name:
                num_samples = input("Количество образцов (по умолчанию 100): ").strip()
                num_samples = int(num_samples) if num_samples.isdigit() else 100
                system.capture_face_data(name, num_samples)
            else:
                print("⚠ Имя не может быть пустым")
        
        elif choice == '3':
            system.list_known_faces()
        
        elif choice == '4':
            name = input("Введите имя для удаления: ").strip()
            if name:
                system.delete_person(name)
        
        elif choice == '5':
            print("До свидания!")
            break
        
        else:
            print("⚠ Неверный выбор. Попробуйте снова.")


if __name__ == "__main__":
    # Проверка наличия opencv-contrib-python
    try:
        cv2.face.LBPHFaceRecognizer_create()
    except AttributeError:
        print("="*60)
        print("ОШИБКА: Требуется opencv-contrib-python")
        print("="*60)
        print("Установите командой:")
        print("pip uninstall opencv-python")
        print("pip install opencv-contrib-python")
        print("="*60)
        exit(1)
    
    main()


СИСТЕМА РАСПОЗНАВАНИЯ ЛИЦ
1. Запустить распознавание в реальном времени
2. Добавить нового человека (обучение)
3. Показать список известных лиц
4. Удалить человека из базы
5. Выход

Захват данных для 'Artem'...
Смотрите в камеру. Будет захвачено 500 кадров.
Медленно поворачивайте голову для лучшего результата.

СИСТЕМА РАСПОЗНАВАНИЯ ЛИЦ
1. Запустить распознавание в реальном времени
2. Добавить нового человека (обучение)
3. Показать список известных лиц
4. Удалить человека из базы
5. Выход
